In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd
import glob
import numpy as np

In [ ]:
data = pd.read_csv('f-speedway.csv')

TARGETS = data.iloc[:,0:3].values
INPUTS = data.iloc[:,3:].values

TARGET_SIZE = TARGETS.shape[1]
INPUT_SIZE = INPUTS.shape[1]
HIDDEN_SIZE = INPUT_SIZE

NUM_LAYERS = 1
BATCH_SIZE = 1
NUM_EPOCHS = 500
LEARNING_RATE = 0.01

def normalize(x, mmin, mmax):
    return clip(x - min(x) )

In [12]:
class LSTMDriver(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, batch_size):
        super(LSTMDriver, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.batch_size = batch_size
        
        super(LSTMDriver, self).__init__()

        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, num_layers=self.num_layers)
        
        self.hidden = self.init_hidden()
        
        self.linear = nn.Linear(self.input_size, self.output_size)

#         self.out = nn.Sigmoid()
        
    def init_hidden(self):
        return (autograd.Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_size), requires_grad=False),
                    autograd.Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_size)), requires_grad=False)
    
    def forward(self, x):
        lstm_out, hidden_out = self.lstm(x.view(1, -1), self.hidden)
        linear_out = self.linear(lstm_out)
        self.hidden = (autograd.Variable(hidden_out[0].data), autograd.Variable(hidden_out[1].data))
#         out = self.out(linear_out)
        
        return linear_out

In [13]:
model = LSTMDriver(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, TARGET_SIZE, BATCH_SIZE)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in np.arange(NUM_EPOCHS):
    loss_val = 0
    for row in data.values:
        model.zero_grad()
        targets = autograd.Variable(torch.Tensor(row[0:3]))
        inputs = autograd.Variable(torch.Tensor(normalize(row[3:], 0, 1))
        
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        loss_val += loss.data[0]
    print('Epoch [%d/%d] | Loss: %f' %(epoch+1, NUM_EPOCHS, loss_val))

Epoch [1/500] | Loss: 5.938199
Epoch [2/500] | Loss: 6.886178
Epoch [3/500] | Loss: 7.286763
Epoch [4/500] | Loss: 13.402437
Epoch [5/500] | Loss: 13.394083
Epoch [6/500] | Loss: 13.391360
Epoch [7/500] | Loss: 13.391040
Epoch [8/500] | Loss: 13.389775
Epoch [9/500] | Loss: 13.389166
Epoch [10/500] | Loss: 13.388746
Epoch [11/500] | Loss: 13.387579
Epoch [12/500] | Loss: 13.386180
Epoch [13/500] | Loss: 13.332379
Epoch [14/500] | Loss: 13.308661
Epoch [15/500] | Loss: 13.300665
Epoch [16/500] | Loss: 13.310490
Epoch [17/500] | Loss: 13.310289
Epoch [18/500] | Loss: 13.300524
Epoch [19/500] | Loss: 13.380461
Epoch [20/500] | Loss: 13.382790
Epoch [21/500] | Loss: 13.382375
Epoch [22/500] | Loss: 13.382201
Epoch [23/500] | Loss: 13.381989
Epoch [24/500] | Loss: 13.381812
Epoch [25/500] | Loss: 13.381651
Epoch [26/500] | Loss: 13.381504
Epoch [27/500] | Loss: 13.381371
Epoch [28/500] | Loss: 13.381249
Epoch [29/500] | Loss: 13.381135
Epoch [30/500] | Loss: 13.381029
Epoch [31/500] | Loss:

Epoch [246/500] | Loss: 13.381778
Epoch [247/500] | Loss: 13.379638
Epoch [248/500] | Loss: 13.380138
Epoch [249/500] | Loss: 13.377541
Epoch [250/500] | Loss: 13.372547
Epoch [251/500] | Loss: 13.372545
Epoch [252/500] | Loss: 13.372542
Epoch [253/500] | Loss: 13.372537
Epoch [254/500] | Loss: 13.372532
Epoch [255/500] | Loss: 13.372520
Epoch [256/500] | Loss: 13.372498
Epoch [257/500] | Loss: 13.372403
Epoch [258/500] | Loss: 13.595721
Epoch [259/500] | Loss: 13.372562
Epoch [260/500] | Loss: 13.372562
Epoch [261/500] | Loss: 13.372562
Epoch [262/500] | Loss: 13.372562
Epoch [263/500] | Loss: 13.372562
Epoch [264/500] | Loss: 13.372562
Epoch [265/500] | Loss: 13.372561
Epoch [266/500] | Loss: 13.372562
Epoch [267/500] | Loss: 13.372561
Epoch [268/500] | Loss: 13.372561
Epoch [269/500] | Loss: 13.372561
Epoch [270/500] | Loss: 13.372562
Epoch [271/500] | Loss: 13.372561
Epoch [272/500] | Loss: 13.372561
Epoch [273/500] | Loss: 13.372561
Epoch [274/500] | Loss: 13.372561
Epoch [275/500

Epoch [487/500] | Loss: 13.372558
Epoch [488/500] | Loss: 13.372559
Epoch [489/500] | Loss: 13.372559
Epoch [490/500] | Loss: 13.372559
Epoch [491/500] | Loss: 13.372560
Epoch [492/500] | Loss: 13.372559
Epoch [493/500] | Loss: 13.372559
Epoch [494/500] | Loss: 13.372559
Epoch [495/500] | Loss: 13.372559
Epoch [496/500] | Loss: 13.372559
Epoch [497/500] | Loss: 13.372559
Epoch [498/500] | Loss: 13.372559
Epoch [499/500] | Loss: 13.372560
Epoch [500/500] | Loss: 13.372559


In [16]:
torch.save(model.state_dict(), 'rnn_params.pt')
torch.save(model, 'whole_net.pt')

/usr/local/anaconda/lib/python3.6/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type LSTMDriver. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
